In [ ]:
%reset -f
# %matplotlib inline
%matplotlib widget

import os
import sys
import re
import glob

from pylab import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import bqplot.pyplot as bplt
import ipywidgets as widgets

from IPython.display import HTML

# from scipy.signal import savgol_filter

import plonk

import ipyvolume as ipv

import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Snapshot

In [ ]:
filename = 'h5_part/binary_00040.h5'
snap = plonk.load_snap(filename)

In [ ]:
snap.available_arrays()

In [ ]:
snap['position']

In [ ]:
# snap.loaded_arrays()
list(snap.properties)

In [ ]:
snap.properties['time']

In [ ]:
# snap.default_units
snap.set_units(time='s', position='km', density='g/cm^3', velocity='km/s')

In [ ]:
snap['x'].max().to('km')

In [ ]:
list(snap._file_pointer['particles'])

In [ ]:
snap['mass'].sum().to('solar_mass')

In [ ]:
snap.image(quantity='gravitational_potential',
    cmap='gist_heat',
    extent=(-50, 50, -50, 50),
    )

In [ ]:
snap['gravitational_potential']

In [ ]:
ax = snap.plot(
    x='x',
    y='gravitational_potential'
)

# Time series

In [ ]:
ts = plonk.load_time_series('binary01.ev')
ts

In [ ]:
# ts['time']
ts.plot('time', ['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z'])

In [ ]:
ts = plonk.load_time_series('binary_inertia.dat')
# ts

In [ ]:
ts.plot('time', ['v1,1'])
# ts.plot('time', ['ang v3'])

# Ptofiles

In [ ]:
prof = plonk.load_profile(snap)

In [ ]:
prof

In [ ]:
prof.loaded_profiles()

In [ ]:
prof.available_profiles()

In [ ]:
prof['angular_momentum_mag']

In [ ]:
snap.plot(x='x', y='gravitational_potential')
prof.plot(x='x', y='gravitational_potential')

# Simulations

In [ ]:
snaps = plonk.load_simulation(prefix='binary', directory='h5_part')

In [ ]:
# snaps.snaps
times = snaps.properties['time']
times

snaps.time_series['global']

In [ ]:
snaps.code_units

In [ ]:
units={'position': 'km', 'density': 'g/cm^3', 'velocity': 'km/s'}
anim = plonk.visualize.animation_particles(
    filename='animation_particles.mp4',
    snaps=snaps.snaps,
    x='x',
    y='y',
    units=units,
    xlim=(-50, 50),
    ylim=(-50, 50),
    text=times,
    adaptive_limits=False,
    save_kwargs={'fps': 24, 'dpi': 600},
)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
snaps.properties

In [ ]:
profiles = list()
for snap in snaps.snaps:
    profiles.append(plonk.load_profile(snap))

In [ ]:
units={'position': 'km', 'density': 'g/cm^3', 'velocity': 'km/s'}
anim = plonk.visualize.animation_profiles(
    filename='animation_profiles.mp4',
    profiles=profiles,
    x='x',
    y='gravitational_potential',
    units=units,
#     xlim=(-50, 50),
#     ylim=(-50, 50),
    adaptive_limits=False,
    save_kwargs={'fps': 24, 'dpi': 600},
)

In [ ]:
HTML(anim.to_html5_video())

# Ipyvolume

In [ ]:
ts = plonk.load_time_series('binary_inertia.dat')
ts

In [ ]:
utime = snaps.code_units['time'].magnitude

# Example snap
snap = snaps.snaps[70]
snap.set_units(time='s', position='km', density='g/cm^3', velocity='km/s')
time_snap = snap.properties['time'].magnitude
time_snap_utime = time_snap/utime

tt = ts.loc[(ts['time'] >= time_snap_utime*0.9999) & (ts['time'] <= time_snap_utime*1.0001)]

tt

In [ ]:
fig = ipv.figure(width=600, height=600)

stars = [snap['x'].magnitude, snap['y'].magnitude, snap['z'].magnitude]

eigenvector = [tt['v1,1'].values, tt['v1,2'].values, tt['v1,3'].values]
t = np.linspace(-40, 40, 100)
x = t*eigenvector[0]
y = t*eigenvector[1]
z = t*eigenvector[2]

pStars = ipv.scatter(stars[0], stars[1], stars[2], size=0.2, color='#1f77b4')
# pStars = ipv.scatter(stars[0], stars[1], snap['gravitational_potential'].magnitude/1e41, size=2., color='#1f77b4')
# pStars = ipv.quiver(stars[0], stars[1], stars[2], snap['velocity_x'].magnitude, snap['velocity_y'].magnitude, snap['velocity_z'].magnitude)

pLine = ipv.plot(x, y, z, color='black')

L1 = [tt['L1,1'].values, tt['L1,2'].values, tt['L1,3'].values]
pL1 = ipv.scatter(L1[0], L1[1], L1[2], size=2., marker='sphere', color='red')
# pL1 = ipv.scatter(L1[0], L1[1], snap['gravitational_potential'].magnitude/1e41, size=2., marker='sphere', color='red')

pCm = ipv.scatter(np.array([0.]), np.array([0.]), np.array([0.]), size=2., marker='sphere', color='orange')

ipv.xyzlim(40)
# ipv.zlim(0,-5)

# ipv.style.axes_off()
ipv.show()

In [ ]:
def lineIn3D(array):
    l = array.shape[0]
    ar = np.identity(l)*array
    x_,y_ = np.mgrid[0:l:1, -40:40:0.8]
    return np.dot(y_.T,ar).T

def pointIn3D(array):
    return np.reshape(array, (-1, 1))

In [ ]:
pos = snaps.to_array(quantity='position').to('km').magnitude
pos[70,:,:] = snaps.to_array(quantity='position').magnitude[70]
# poten = snaps.to_array(quantity='gravitational_potential').magnitude/1e41

fig = ipv.figure(width=600, height=600)

x = pos[:,:,0]
y = pos[:,:,1]
z = pos[:,:,2]
# z = poten[:,:]
pStars = ipv.scatter(x, y, z, size=0.2, color='#1f77b4')

x1 = lineIn3D(ts['v1,1'].values)
y1 = lineIn3D(ts['v1,2'].values)
z1 = lineIn3D(ts['v1,3'].values)

pLine = ipv.plot(x1, y1, z1, color='black')

x2 = pointIn3D(ts['L1,1'].values)
y2 = pointIn3D(ts['L1,2'].values)
z2 = pointIn3D(ts['L1,3'].values)

# pL1 = ipv.scatter(ts['L1,1'].values, ts['L1,2'].values, ts['L1,3'].values, size=2., marker='sphere', color='red')
pL1 = ipv.scatter(x2, y2, z2, size=2., marker='sphere', color='red')

pCm = ipv.scatter(np.array([0.]), np.array([0.]), np.array([0.]), size=2., marker='sphere', color='orange')

ipv.xyzlim(40)
# ipv.zlim(0,-5)
ipv.animation_control((pStars,pLine,pL1,pCm)) # shows controls for animation controls
# ipv.animation_control(scatter) # shows controls for animation controls

ipv.show()

In [ ]:
ipv.view(210, 150, distance=2.3)

In [ ]:
# This is commented out, otherwise it would run on readthedocs
def set_view(figure, framenr, fraction):
    ipv.view(210, 150, distance=2.3)
#     scatter.size = size * (2+0.5*np.sin(fraction * 6 * np.pi))
ipv.movie('wave.gif', set_view, fps=20, frames=20*5, gif_loop=True)

# Gravitational potential

In [ ]:
pos = snaps.to_array(quantity='position').to('km').magnitude
pos[70,:,:] = snaps.to_array(quantity='position').magnitude[70]

line = np.array([ts['v1,1'].values, ts['v1,2'].values, ts['v1,3'].values])
# line.shape
# pos[:,:,:].shape

def update_plot(message):
#     print(message)
    if message['name'] == 'value':
        time = slider.value
        scat.x = np.dot(line[:,time],pos[time,:,:].T)
        scat.y = snaps.snaps[time]['gravitational_potential'].magnitude/1e41

noise = 2
x_ = np.dot(line[:,0],pos[0,:,:].T)
y_ = snaps.snaps[0]['gravitational_potential'].magnitude/1e41

figure = bplt.figure(animation_duration=10)
#figure.animation_duration = 250
scat = bplt.scatter(x_, y_)
scat.observe(update_plot, ['x','y'])

slider = widgets.IntSlider(description='time', value=0, min=0, max=len(ts)-1)
slider.observe(update_plot, 'value')

widgets.VBox([slider, figure])

In [ ]:
poten = pd.read_csv('fort.10000', header=None, delim_whitespace=True)

# p = poten.loc[poten[0] == 90.0]
p = poten
# p = p.loc[:,(p.columns != 0) & (p.columns != 5)].T.values
p = p.loc[:,(p.columns != 0)].T.values

# poten2 = pd.read_csv('fort.10001', header=None, delim_whitespace=True)
# p2 = poten2
# p2 = p2.loc[:,(p2.columns != 0)].T.values

In [ ]:
fig = plt.figure()

# poten
plt.scatter(p[4], p[3]/1.4e-7)
# plt.plot(p2[3], p2[-1]/2e-2)
# plt.scatter(p2[3], p2[4]/1e-3)

# force + -poten
# plt.scatter(p[4], -p[3]/7.0e-5)
# plt.scatter(p2[3], p2[4])
# plt.scatter(p2[3], p2[5])
# plt.ylim(-1e-2,1e-2)

# log10
# plt.scatter(p[4], np.log10(-p[3]))
# plt.plot(p2[3], np.log10(-p2[-1]))
# plt.scatter(p2[3], np.log10(p2[4]))
# plt.scatter(p2[3], np.log10(p2[5]))

# Angular momentum

In [ ]:
bc = plonk.load_time_series('binary_centres.dat')
bc

In [ ]:
# bc.plot('time', ['n L_tot'])

bc2 = bc.loc[bc['n L_tot'] != 0]

fig = plt.figure()
plt.rcParams.update(
    {
    "text.usetex": False,
#     "font.family": "serif",
#     "font.sans-serif": ["Computer Modern"]
    }
)
plt.plot(bc2['time'].values*utime, (bc2['n L_tot'].values-bc2['n L_tot'].values[0]), label=r'L_{tot}(t)-L_tot(t_0)')
plt.plot(bc2['time'].values*utime, (bc2['n L_A'].values-bc2['n L_A'].values[0]), label=r'L_1(t)-L_1(t_0)')
plt.plot(bc2['time'].values*utime, (bc2['n L_B'].values-bc2['n L_B'].values[0]), label=r'L_2(t)-L_2(t_0)')

plt.xlabel('time [s]')

plt.legend()